In [13]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

lr = 0.01
EPOCHS = 100

# data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=141
)

sc = StandardScaler()
x_train = sc.fit_transform(X_train)
x_test = sc.transform(X_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

# model
# f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        
        # layers
        self.linear = nn.Linear(n_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
model = LogisticRegression(n_features)

# loss and optimizer
loss = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr)

# training
for epoch in range(EPOCHS):
    # 1. forward pass & loss
    y_pred = model(x_train)
    l = loss(y_pred, y_train)
    
    # 2. backward pass
    l.backward()
    
    # 3. gradients
    optimizer.step()
    
    # empty gradients
    optimizer.zero_grad()
    
    if (epoch+1)%10 == 0:
        print(f'epoch: {epoch + 1}, loss = {l.item():.4f}')

with torch.no_grad():
    y_predicted = model(x_test)
    # need to convert to 0 or 1
    y_predicted_class = y_predicted.round()
    acc = y_predicted_class.eq(y_test).sum()/float(y_test.shape[0])
    
    print(f'accuracy = {acc:.4f}')

epoch: 10, loss = 0.7064
epoch: 20, loss = 0.5649
epoch: 30, loss = 0.4779
epoch: 40, loss = 0.4197
epoch: 50, loss = 0.3781
epoch: 60, loss = 0.3468
epoch: 70, loss = 0.3222
epoch: 80, loss = 0.3023
epoch: 90, loss = 0.2859
epoch: 100, loss = 0.2719
accuracy = 0.9298


In [11]:
y_test.size()

torch.Size([114, 1])